# Amazon Transcribe と Amazon Comprehend を boto3 で使用
本ノートブックでは、AWS SDK for Python boto3 を用いて、下記操作を行います。

1. 音声ファイル(sample.mp3) を Amazon Transcribe の start_transcription_job API を用いて、文字起こしジョブを起動
2. 1 で作成した文字起こし結果に対して、Amazon Comprehend で解析を行います。
  1. 言語検出(detect_dominant_language)
  2. エンティティ認識(detect_entities)
  3. キーフレーズ抽出(detect_key_phrases)
  4. 感情分析(detect_sentiment)

本ノートブックのハンズオンは各セルを全て実行するだけで完了できます。

---

## Jupyter Notebook を使用したことが無い方

* 各セルに Python のコードが書かれております。Shift + Enter キーでそのコードを実行することができます。
* メニューの Cell から Run Cells を実行してもよいです。
---
最初に使用するライブラリを読み込みます


In [ ]:
# 使用するライブラリの読み込み
import boto3,sagemaker,json
from time import sleep

* AWS SDK for Python で AWS のサービスを扱うには、サービスごとにクライアントを生成し、そのクライアントが持つメソッドを実行していきます
* 本ノートブックでは、Amazon Transcribe, Amazon Comprehend, S3 の3つを使用するため、それらのクライアントを生成します

In [ ]:
# AWS サービス の API を実行するためのクライアントを生成
transcribe = boto3.client('transcribe')
comprehend = boto3.client('comprehend')
s3 = boto3.client('s3')

# Amazon Transcribe

* Amazon Transcribe で文字起こしするには、音声ファイルを S3 というストレージサービスにアップロードする必要があります
* S3 クライアントを利用して sample.mp3 ファイルを S3 にアップロードします
* このセルを実行したあと、ぜひ AWS マネジメントコンソールにて、S3 にアップロードされているかをご確認ください

In [ ]:
# 音声ファイル (sample.mp3) を s3 にアップロード
bucket = sagemaker.session.Session().default_bucket()
mp3file = 'sample.mp3'
input_key_prefix = 'transcribe/input/'
input_key = input_key_prefix+mp3file
s3.upload_file(mp3file,bucket,input_key)
input_s3uri = 's3://'+bucket+'/' + input_key


transcribe_job_name = 'sample'
output_key = 'transcribe/output/'+transcribe_job_name + '.json'

* Amazon Transcirbe の文字起こしジョブを開始します
* 必須入力項目は以下3つです
    * TranscriptionJobName
    * LanguageCode
    * Media
* OutputBucketName/OutputKey については必須ではないですが、指定しないと Amazon Transcribe サービスが保有する S3 バケットに出力され、15 分間で揮発する署名付き URL からしかダウンロードできないため、明示的に指定しております

詳しくは[こちら](https://boto3.amazonaws.com/v1/documentation/api/latest/reference/services/transcribe.html#TranscribeService.Client.start_transcription_job)をご参照ください。

---

前処理として、ジョブ名が sample という文字起こしジョブがあるかをチェックし、存在したら削除する、という処理が入っています。これは、同一のジョブ名を複数起動できないため、同じパラメータで複数回実行できないのを回避するためです。

In [ ]:
# 音声ファイルを文字起こし
if len(transcribe.list_transcription_jobs(JobNameContains=transcribe_job_name)['TranscriptionJobSummaries'])==1:
    transcribe.delete_transcription_job(TranscriptionJobName=transcribe_job_name)
    sleep(1)
transcribe.start_transcription_job(
    TranscriptionJobName=transcribe_job_name,
    LanguageCode='ja-JP',
    Media={
        'MediaFileUri': input_s3uri
    },
    OutputBucketName=bucket,
    OutputKey=output_key
)

start_transcribe_job は非同期処理ですので、上のセルの実行が完了した段階では処理が完了しているとは限りません。list_transcription_jobs から 先程実行した transcription job の status を 1 秒おきに取得し、完了を待ちます

In [ ]:
while True:
    status = transcribe.list_transcription_jobs(JobNameContains=transcribe_job_name)['TranscriptionJobSummaries'][0]['TranscriptionJobStatus']
    if status == 'COMPLETED':
        print('.')
        print('COMPLETED!')
        break
    elif status == 'FAILED':
        print('.')
        print('FAILED')
        break
    else:
        print('.',end='')
        sleep(1)

文字起こし結果は、S3 の中に {ジョブ名}.json として格納されますので、ノートブックインスタンスに S3 からダウンロードしてきます。

In [ ]:
s3.download_file(bucket,output_key, transcribe_job_name+'.json')

文字起こし結果を確認します。下のセルを実行すると文字起こし結果を確認することができます。

In [ ]:
with open('sample.json','rt') as f:
    transcription_json = json.loads(f.read())
transcript_txt = transcription_json['results']['transcripts'][0]['transcript'].replace(' ','').replace('、','')
print('文字起こし結果: '+ transcript_txt)

# Amazon Comprehend
文字起こしの結果を分析していきます。どの API も分析したいテキストを引数に入れて call します

言語検出は detect_dominant_language で実行します。日本語であることが確認取れます。
詳細は[こちら](https://boto3.amazonaws.com/v1/documentation/api/latest/reference/services/comprehend.html#Comprehend.Client.detect_dominant_language)をご参照ください

In [ ]:
comprehend.detect_dominant_language(Text = transcript_txt)

エンティティ認識は detect_entities で実行します。以降の API は言語を指定する必要があります。
詳細は[こちら](https://boto3.amazonaws.com/v1/documentation/api/latest/reference/services/comprehend.html#Comprehend.Client.detect_entities)をご参照ください。

In [ ]:
comprehend.detect_entities(Text = transcript_txt, LanguageCode= 'ja')

キーフレーズ抽出は detect_key_phrases で実行します。詳細は[こちら](https://boto3.amazonaws.com/v1/documentation/api/latest/reference/services/comprehend.html#Comprehend.Client.detect_key_phrases)をご参照ください

In [ ]:
comprehend.detect_key_phrases(Text = transcript_txt, LanguageCode='ja')

感情分析は detect_sentiment で実行します。詳細は[こちら](https://boto3.amazonaws.com/v1/documentation/api/latest/reference/services/comprehend.html#Comprehend.Client.detect_sentiment)をご参照ください

In [ ]:
comprehend.detect_sentiment(Text = transcript_txt, LanguageCode = 'ja')